In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
base_dir = r'C:\Users\USER\dev\mechanics'
csv_file = 'blade_profile.csv'
csv_dir = os.path.join(base_dir, csv_file)

In [5]:
df = pd.read_csv(csv_dir)

In [6]:
dfdict = df.to_dict()

In [7]:
for key, value in dfdict.items():
    print(key)

r_position
wind_velocity
pitch_angle
chord_length


# make arrays and insert to df

In [8]:
df = pd.read_csv(csv_dir)

In [9]:
dfdict = df.to_dict(orient='index')

In [10]:
from xfoil import XFoil
xf = XFoil()
array6409 = np.loadtxt("NACA6409.dat", skiprows=1)
array2412 = np.loadtxt("NACA2412.dat", skiprows=1)
from xfoil.model import Airfoil
naca6409 = Airfoil(x=array6409[:,0], y=array6409[:,1])
naca2412 = Airfoil(x=array2412[:,0], y=array2412[:,1])

In [13]:
12*array6409[:,0]

array([1.200000e+01, 1.191360e+01, 1.175868e+01, 1.156224e+01,
       1.133460e+01, 1.108200e+01, 1.080900e+01, 1.051896e+01,
       1.021476e+01, 9.898680e+00, 9.572880e+00, 9.238920e+00,
       8.898480e+00, 8.553000e+00, 8.203800e+00, 7.852080e+00,
       7.499040e+00, 7.145640e+00, 6.792840e+00, 6.441600e+00,
       6.092880e+00, 5.747400e+00, 5.406000e+00, 5.069520e+00,
       4.738560e+00, 4.413840e+00, 4.096200e+00, 3.786120e+00,
       3.484200e+00, 3.191280e+00, 2.907720e+00, 2.634120e+00,
       2.371200e+00, 2.119320e+00, 1.879200e+00, 1.651200e+00,
       1.435800e+00, 1.233600e+00, 1.045080e+00, 8.707200e-01,
       7.108800e-01, 5.660400e-01, 4.366800e-01, 3.232800e-01,
       2.262000e-01, 1.458000e-01, 8.256000e-02, 3.696000e-02,
       9.360000e-03, 0.000000e+00, 9.360000e-03, 3.696000e-02,
       8.256000e-02, 1.458000e-01, 2.262000e-01, 3.232800e-01,
       4.366800e-01, 5.660400e-01, 7.108800e-01, 8.707200e-01,
       1.045080e+00, 1.233600e+00, 1.435800e+00, 1.6512

In [11]:
for key, value in dfdict.items():
    print(key)

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [22]:
import math
w = 30
for key, value in dfdict.items():
    value["blade_velocity"] = value['r_position'] * w
    value["relative_velocity"] = round(math.sqrt(value["blade_velocity"]**2+value["wind_velocity"]**2),2)
    value["arctan"] = math.degrees(math.atan2(value["wind_velocity"], value["blade_velocity"]))
    value["angle_of_attack"] = round(value["arctan"] - value["pitch_angle"])
    value["Reynolds_number"] = round(value["relative_velocity"] * value["chord_length"] / 0.00001511, 2)

    xf = XFoil()
    if key < 13: 
        xf.airfoil = naca6409
    else:
        xf.airfoil = naca2412
    xf.Re = value["Reynolds_number"]
    xf.max_iter = 40
    print(xf.a(value["angle_of_attack"]))

(1.1684448719024658, 0.15271762013435364, -0.09518875181674957, -3.226717233657837)
(4.5795488357543945, 0.24444781243801117, -1.0503182411193848, -6.537448883056641)
(15.316740036010742, 0.00231098267249763, -4.012783527374268, -18.068981170654297)
(7.4373321533203125, 0.024582499638199806, -1.2668917179107666, -19.731874465942383)
(10.52900505065918, 0.014552026987075806, -2.114168643951416, -25.917217254638672)
(13.557140350341797, 0.007764446083456278, -2.947007894515991, -31.858436584472656)
(24.578046798706055, 0.0028934190049767494, -6.1869120597839355, -43.95894241333008)
(26.759639739990234, 0.0040102736093103886, -6.78228235244751, -49.82451248168945)
(26.39944076538086, 0.004681747406721115, -6.6490068435668945, -53.06563949584961)
(31.966272354125977, 0.00436283927410841, -8.365006446838379, -62.57804489135742)
(29.982906341552734, 0.004469072911888361, -7.684476852416992, -65.11679077148438)
(42.18291473388672, 0.001904271193780005, -11.703627586364746, -76.86214447021484)

In [30]:
w = 40
for key, value in dfdict.items():
    value["blade_velocity"] = value['r_position'] * w
    value["relative_velocity"] = round(math.sqrt(value["blade_velocity"]**2 + value["wind_velocity"]**2),2)
    value["arctan"] = math.degrees(math.atan2(value["wind_velocity"], value["blade_velocity"]))
    aoa = round(value["arctan"] - value["pitch_angle"], 2)
    value["angle_of_attack"] = aoa
    re_n = round(value["relative_velocity"] * value["chord_length"] / 0.00001511, 2)
    value["Reynolds_number"] = re_n
    xf = XFoil()
    if key < 13: 
        xf.airfoil = naca6409
    else:
        xf.airfoil = naca2412
    xf.Re = re_n
    xf.max_iter = 400
    value["Cl"], value["Cd"], value["Cm"], value["Cp"] = xf.a(aoa)

In [31]:
pd.DataFrame.from_dict(dfdict, orient="index")

,r_position,wind_velocity,pitch_angle,chord_length,blade_velocity,relative_velocity,arctan,angle_of_attack,Reynolds_number,Cl,Cd,Cm,Cp
0,0.0250,19.03,73,0.038,1.0,19.06,86.991953,13.99,47933.82,1.433684,0.100463,-0.061428,-4.794329
1,0.0375,18.43,70,0.051,1.5,18.49,85.347008,15.35,62408.34,0.991816,0.108312,-0.046062,-3.372934
2,0.0500,17.79,67,0.065,2.0,17.90,83.585586,16.59,77001.99,0.822198,0.163458,-0.079447,-1.888513
3,0.0625,17.09,64,0.074,2.5,17.27,81.677552,17.68,84578.42,1.145208,0.235852,-0.140462,-2.184724
4,0.0750,16.34,61,0.082,3.0,16.61,79.596442,18.60,90140.30,1.176231,0.250353,-0.144559,-2.336841
5,0.0875,15.54,58,0.086,3.5,15.93,77.307334,19.31,90667.11,1.178997,0.261760,-0.155873,-2.201487
6,0.1000,14.69,55,0.088,4.0,15.22,74.767986,19.77,88640.64,0.596212,0.211964,-0.084663,-0.762847
7,0.1125,13.78,51,0.086,4.5,14.50,71.915021,20.92,82528.13,1.225243,0.288973,-0.168498,-2.216424
8,0.1250,12.82,47,0.082,5.0,13.76,68.693441,21.69,74673.73,0.824403,0.248640,-0.111414,-1.269240
9,0.1375,11.81,42,0.074,5.5,13.03,65.028274,23.03,63813.37,0.836017,0.269033,-0.117737,-1.189329


In [14]:
df2= pd.DataFrame.from_dict(dfdict, orient="index")

In [15]:
df2

,r_position,wind_velocity,pitch_angle,chord_length,blade_velocity,relative_velocity,arctan,angle_of_attack,Reynolds_number
0,0.0250,19.03,73,0.038,0.750,19.04,87.743058,15,51000.0
1,0.0375,18.43,70,0.051,1.125,18.46,86.506898,17,66000.0
2,0.0500,17.79,67,0.065,1.500,17.85,85.180390,18,81000.0
3,0.0625,17.09,64,0.074,1.875,17.19,83.738933,20,89000.0
4,0.0750,16.34,61,0.082,2.250,16.49,82.159739,21,95000.0
5,0.0875,15.54,58,0.086,2.625,15.76,80.412167,22,95000.0
6,0.1000,14.69,55,0.088,3.000,14.99,78.457738,23,92000.0
7,0.1125,13.78,51,0.086,3.375,14.19,76.238014,25,85000.0
8,0.1250,12.82,47,0.082,3.750,13.36,73.695184,27,77000.0
9,0.1375,11.81,42,0.074,4.125,12.51,70.746699,29,65000.0


In [365]:
1e6

1000000.0